In [1]:
import pandas as pd
import redis
import json
import time

In [2]:
df_carList = pd.read_csv("./datasets/Car_listings.csv", delimiter=",")

In [3]:
df_carList = df_carList.drop(['Price'], axis = 1)

In [4]:
df_carList.to_json('./datasets/cars.json', orient='split', compression='infer', index='true')

In [5]:
df = pd.read_json('./datasets/cars.json', orient ='split', compression = 'infer')

In [7]:
r = redis.Redis(db=2)

In [8]:
with open('./datasets/cars.json') as json_file:
    data = json.load(json_file)
    print("\nData:", data['data'][0])


Data: [2014, 35725, 'El Paso', ' TX', '19VDE2E53EE000083', 'Acura', 'ILX6-Speed']


In [ ]:
listOfDict = []
for dic in data['data']:
    dicti = dict(zip(list(df), dic))
    listOfDict.append(dicti)
    if (data['data'].index(dic)+1 == 200000):
        break    # break here
print(len(listOfDict))

In [15]:
index_list = [*range(1, (len(listOfDict)+1), 1)]
print(len(index_list))

852122


In [17]:
fullDict = dict(zip(index_list, listOfDict))

In [19]:
with open("./datasets/carsDict.json", "w") as write_file:
    json.dump(fullDict, write_file, indent=4)

In [20]:
with open('./datasets/carsDict.json') as json_file:
    data = json.load(json_file)

In [21]:
with r.pipeline() as pipe:
    for dic in data:
        #print(dic)
        #print(data[dic])
        pipe.hmset(dic, data[dic])
        pipe.execute()
r.bgsave()

C:\Users\miade\AppData\Local\Temp/ipykernel_5884/3408369710.py:5: DeprecationWarning: Pipeline.hmset() is deprecated. Use Pipeline.hset() instead.
  pipe.hmset(dic, data[dic])


True